<a href="https://colab.research.google.com/github/Greqorian/Ebay_kleineanzeige_img_scraper/blob/main/Kleineanzeige_Images_Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
# package imports
#basics
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import json # json files

#scrapping
from bs4 import BeautifulSoup
import requests
from datetime import datetime


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

# Nimmt sich das momentane Datum & formatiert es zu '[Stunde:Minute:Sekunde] - '.
now = datetime.now()
prefix = "[" + now.strftime("%H:%M:%S") + "] - "

# Eine Funktion um eine Text-Datei zu beschreiben.
def writeFile(response, name ,fileType):
    with open(name + fileType, "a", encoding="utf8") as f:
        f.write(response)




In [ ]:
def getSoupFromUtl(url):
   # Setzt die Headers der Anfrage (Den User-Agent), damit Ebay-Kleinanzeigen die Anfrage nicht blockt.
  headers = {
      'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36 Edg/84.0.522.59',
  }

  # Gibt den HTML text der Website in eine Variable wieder.
  response = requests.get(url, headers=headers)

  # Setzt den Content der Website in eine Variable.
  page = response.content

  # Erstellt eine BeautifulSoup-Instanz mit dem Website-Content und einem passendem Parser.
  soup = BeautifulSoup(page, "html.parser")

  return soup

In [ ]:
# returns the list of all url adress pages from kleineanzeige search
def getItemsUlrsList(itemName):

  # Fragt die Query ab die Später in den URL gegeben werden soll (der Suchbegriff)
  query = "Ikea-" + itemName

  # Fügt die Query in den Ebay-Kleinanzeigen URL ein.
  URL = "https://www.ebay-kleinanzeigen.de/s-" + query + "/k0"

  # Gibt aus welche Query gewählt wurde.
  # print(prefix + "Es wird nach gesucht>> " + query)

  soup = getSoupFromUtl(URL)

  # Setzt die Search-Results-Content in eine Variable.
  srchRsltsContent = soup.find("div", id="srchrslt-content")

  # Setzt die Einträge inheralb der Seach-Results und dem Table dortdrinn in eine Variable / Array.
  srchRslts = srchRsltsContent.find_all('a', href=True)

  itemPagesUrls = []

  # save a list of subpages URLs
  for a in srchRslts:
      
      if '/s-anzeige' in a['href']:
        itemPagesUrls.append('https://www.ebay-kleinanzeigen.de' + a['href'])

  return itemPagesUrls


In [ ]:
# returns a list of sources for all item images
def getItemsImagesSrcList(itemPagesUrls):

  imagesSrcList = []

  for URL in itemPagesUrls:
    
    soupForSubPage = getSoupFromUtl(URL)

    srchRslts2 = soupForSubPage.find_all("div", {"class": "galleryimage-element"})

    for tag in srchRslts2:
      image = tag.find("img")
      if hasattr(image, 'src'):
        imagesSrcList.append(image['src'])

  return imagesSrcList


In [ ]:
# Opening JSON file
f = open('/content/drive/MyDrive/DATA/IKEAProductLists/ikeaRangeList.json', "r")
# a dictionary
ikeaRangeListJson = json.load(f)

print(len(ikeaRangeListJson))


5


In [ ]:
print(ikeaRangeListJson)

[{'id': '60510640', 'name': 'JÄRVFJÄLLET', 'title': 'JÄRVFJÄLLET Office chair with armrests', 'url': 'https://www.ikea.com/us/en/p/jaervfjaellet-office-chair-with-armrests-60510640/', 'currency': 'USD', 'price': '279', 'category': 'Furniture', 'group': 'Chairs', 'subgroup': 'Desk chairs'}, {'id': '49324512', 'name': 'ÄPPLARÖ', 'title': 'ÄPPLARÖ Table and 4 folding chairs, outdoor - brown stained/Frösön/Duvholmen dark gray', 'url': 'https://www.ikea.com/us/en/p/aepplaroe-table-and-4-folding-chairs-outdoor-brown-stained-froesoen-duvholmen-dark-gray-s49324512/', 'currency': 'USD', 'price': '268.96', 'category': 'Furniture', 'group': 'Patio furniture', 'subgroup': 'Outdoor dining sets'}, {'id': '19331892', 'name': 'LOBERGET', 'title': 'LOBERGET / BLYSKÄR Swivel chair with pad - white/blue', 'url': 'https://www.ikea.com/us/en/p/loberget-blyskaer-swivel-chair-with-pad-white-blue-s19331892/', 'currency': 'USD', 'price': '34.99', 'category': 'Furniture', 'group': 'Chairs', 'subgroup': 'Desk ch

In [ ]:
itemsList = []

for item in ikeaRangeListJson:
  itemsList.append({'id': item['id'], 'name': item['name'], 'urls': getItemsUlrsList(item['name']), 'imgSrcs': []})

print(itemsList) 

AttributeError: ignored

In [ ]:
print(len(itemsUrlsList))

imagesSrc = getItemsImagesSrcList(itemsUrlsList)

print(len(imagesSrcList))
print(imagesSrcList)

226


KeyboardInterrupt: ignored

In [ ]:
os.chdir('/content/drive/MyDrive/DATA')

# Check current working directory.
curDir = os.getcwd() 
  # print("Current working directory:", curDir)
  # try:
#os.mkdir(os.path.join(curDir, 'IKEAProductLists'))
  # except:
  #     pass

os.chdir(os.path.join(curDir, 'IKEAProductLists'))
!pwd

/content/drive/MyDrive/DATA/IKEAProductLists


Save the array of IKEA Products to JSON file

2. Create Images list from the Products List

In [ ]:
# Opening JSON file
f = open('/content/drive/MyDrive/DATA/IKEAProductLists/ikeaProducts.json', "r")
# a dictionary
ikeaProducts = json.load(f)
print(len(ikeaProducts))

11279


In [ ]:
ikeaImagesList = []
range = [0,5]

for x in selectedCategoryList[range[0]:range[1]]:
  print(x['url'])

  url = x['url']
  id = x['id']
  name = x['name']
  title = x['title'].replace(' ', '_').replace('/', '-').replace(',', '').replace('"', '')
  category = x['category']
  group = x['group']
  subgroup = x['subgroup']

  r = requests.get(url)
  soup = BeautifulSoup(r.text, 'html.parser')
  images = soup.find_all('img')

  for image in images:

    if hasattr(image, 'alt') and hasattr(image, 'src') and name in image['alt']:

      ikeaImagesList.append({
          'id': id, 
          'name': name, 
          'title': title, 
          'category':category, 
          'group':group, 
          'subgroup':subgroup, 
          'src': image['src']  
          })
      
  print(len(ikeaImagesList))    

https://www.ikea.com/us/en/p/jaervfjaellet-office-chair-with-armrests-60510640/
9
https://www.ikea.com/us/en/p/aepplaroe-table-and-4-folding-chairs-outdoor-brown-stained-froesoen-duvholmen-dark-gray-s49324512/
14
https://www.ikea.com/us/en/p/loberget-blyskaer-swivel-chair-with-pad-white-blue-s19331892/
22
https://www.ikea.com/us/en/p/bekant-sit-stand-underframe-for-table-top-white-80263219/
25
https://www.ikea.com/us/en/p/langfjaell-office-chair-with-armrests-gunnared-beige-black-s79177859/
38
